In [1]:
from os.path import join
import pandas as pd
import re
from spacy.symbols import IDS
from constants import *

In [2]:
fname = 'OnlineParticipation_nlp.pickle'
#Europarl_nlp.pickle
#FAZ_nlp.pickle
#FOCUS_nlp.pickle
#PoliticalSpeeches_nlp.pickle
fpath = join(NLP_PATH, fname)
df = pd.read_pickle(fpath)
df.shape

(1725466, 10)

In [3]:
df

,hash,tok_idx,sent_idx,text,token,POS,ent_iob,ent_idx,ent_type,noun_phrase
0,8109220208026988884,0,0,Nebentätigkeiten,Nebentätigkeit,NOUN,O,0,,1
1,8109220208026988884,1,0,von,von,ADP,O,0,,0
2,8109220208026988884,2,0,OB,OB,NOUN,B,1,ORG,2
3,8109220208026988884,3,0,und,und,CONJ,O,0,,0
4,8109220208026988884,4,0,Kommunalpoilter,Kommunalpoilter,NOUN,B,2,ORG,3
5,8109220208026988884,5,0,-,-,PUNCT,O,0,,0
6,8109220208026988884,6,0,öffentliches,öffentlich,ADJ,O,0,,4
7,8109220208026988884,7,0,Ehrenamt,Ehrenamt,NOUN,O,0,,4
8,8109220208026988884,8,0,\n,,SPACE,B,3,ORG,0
9,8109220208026988884,9,1,Durch,durch,ADP,O,0,,0


In [4]:
def concat_entities(column):
    if column.name in {HASH, SENT_IDX, ENT_IDX, ENT_TYPE}:
        return column.values[0]
    if column.name in {'tok_idx', NOUN_PHRASE, 'np'}:
        return tuple(column.values)
    if column.name in {TEXT, TOKEN}:
        return column.str.cat(sep='_')
    return False

In [5]:
# extracting spacy NER
df_entities = df.loc[(df[ENT_IDX] > 0) & (df.POS != SPACE)]
df_entities = df_entities.groupby(ENT_IDX).filter(lambda x: len(x) > 1)
df_entities = df_entities.groupby(ENT_IDX, as_index=False).agg(concat_entities)
df_entities['len'] = df_entities.loc[:, TOK_IDX].apply(lambda idxs: len(idxs))
df_entities[POS] = 'NER'
df_entities[ENT_IOB] = 'P'
df_entities[POS] = df_entities[POS].astype("category")
df_entities[ENT_IOB] = df_entities[ENT_IOB].astype("category")
df_entities[ENT_TYPE] = df_entities[ENT_TYPE].astype("category")
df_entities

,ent_idx,hash,tok_idx,sent_idx,text,token,POS,ent_iob,ent_type,noun_phrase,len
0,7,-1741504268962868006,"(10, 11)",4,ca.5_Jahren,ca.5_Jahr,NER,P,MISC,"(21, 21)",2
1,12,-8078070548414875519,"(0, 1, 2)",12,Kosteneinsparung_bei_Friedhof-/Parkanlagen,Kosteneinsparung_bei_Friedhof-/Parkanlage,NER,P,MISC,"(38, 0, 39)",3
2,14,-8078070548414875519,"(8, 9)",13,z._B.,z._B.,NER,P,MISC,"(0, 41)",2
3,25,1112788597393434588,"(0, 2, 3)",18,Leerstand_Siehe_Leerstandsmelder,Leerstand_sehen_Leerstandsmelder,NER,P,PER,"(71, 0, 72)",3
4,26,1112788597393434588,"(8, 9)",18,Stadt_Bonn,Stadt_Bonn,NER,P,LOC,"(74, 74)",2
5,29,1112788597393434588,"(21, 22, 23, 24, 25, 26, 27)",20,Ein_Teil_gehört_auch_der_Stadt_Bonn,Ein_Teil_gehören_auch_der_Stadt_Bonn,NER,P,MISC,"(77, 77, 0, 78, 78, 78, 78)",7
6,43,-2263838526400175399,"(55, 56)",34,Bsp_Altweiber,BSP_Altweib,NER,P,PER,"(160, 160)",2
7,52,-2263838526400175399,"(141, 142)",39,SGB_II-Bezieher,SGB_Ii-Bezieher,NER,P,MISC,"(184, 184)",2
8,62,3346603037676320710,"(0, 1)",46,Sparkasse_Köln,Sparkasse_Köln,NER,P,ORG,"(222, 222)",2
9,64,3346603037676320710,"(28, 29)",51,z._T,z._T,NER,P,MISC,"(0, 232)",2


In [6]:
# extracting spacy noun chunks
df_np = df.loc[(df[NOUN_PHRASE] > 0) & (~df[POS].isin({SPACE, NUM, DET, SYM}))]
df_np = df_np.groupby(NOUN_PHRASE).filter(lambda x: len(x) > 1)
df_np = df_np.groupby(NOUN_PHRASE, as_index=False).agg(concat_entities)
df_np['len'] = df_np.loc[:, TOK_IDX].apply(lambda idxs: len(idxs))
df_np[POS] = 'NOUN_PHRASE'
df_np[ENT_IOB] = 'P'
df_np[POS] = df_np[POS].astype("category")
df_np[ENT_IOB] = df_np[ENT_IOB].astype("category")
df_np[ENT_TYPE] = df_np[ENT_TYPE].astype("category")
df_np

,noun_phrase,hash,tok_idx,sent_idx,text,token,POS,ent_iob,ent_idx,ent_type,len
0,4,8109220208026988884,"(6, 7)",0,öffentliches_Ehrenamt,öffentlich_Ehrenamt,NOUN_PHRASE,P,0,,2
1,5,8109220208026988884,"(11, 12)",1,"sogenannte""öffentliche_Ehrenamt","sogenannte""öffentliche_Ehrenamt",NOUN_PHRASE,P,0,,2
2,15,8109220208026988884,"(50, 51, 52)",2,sogenanntes_öffentliches_Ehrenamt,sogenannt_öffentlich_Ehrenamt,NOUN_PHRASE,P,0,,3
3,18,-1741504268962868006,"(1, 2)",3,(_Gabi,(_Gabi,NOUN_PHRASE,P,0,,2
4,21,-1741504268962868006,"(10, 11)",4,ca.5_Jahren,ca.5_Jahr,NOUN_PHRASE,P,7,MISC,2
5,22,-1741504268962868006,"(13, 15)",4,sehr_Geld,sehr_Geld,NOUN_PHRASE,P,0,,2
6,25,-1741504268962868006,"(27, 28)",5,neue_Wache,neu_Wache,NOUN_PHRASE,P,0,,2
7,33,-5819192628571290874,"(11, 13, 15, 16)",11,als_-_%_Zinsen,als_-_%_Zins,NOUN_PHRASE,P,0,,4
8,35,-5819192628571290874,"(21, 22)",12,neuer_Kredite,neu_Kredit,NOUN_PHRASE,P,0,,2
9,36,-5819192628571290874,"(25, 26, 27)",12,derzeit_niedrigen_Zinsen,derzeit_niedrig_Zins,NOUN_PHRASE,P,0,,3


In [7]:
# intersecting both extraction methods returns very nice results
df_new = df_entities.append(df_np)
dups = df_new.duplicated([HASH, SENT_IDX, TOK_IDX])
df_dups = df_new[dups]
df_dups

,POS,ent_idx,ent_iob,ent_type,hash,len,noun_phrase,sent_idx,text,tok_idx,token
4,NOUN_PHRASE,7,P,MISC,-1741504268962868006,2,21,4,ca.5_Jahren,"(10, 11)",ca.5_Jahr
19,NOUN_PHRASE,26,P,LOC,1112788597393434588,2,74,18,Stadt_Bonn,"(8, 9)",Stadt_Bonn
36,NOUN_PHRASE,43,P,PER,-2263838526400175399,2,160,34,Bsp_Altweiber,"(55, 56)",BSP_Altweib
41,NOUN_PHRASE,52,P,MISC,-2263838526400175399,2,184,39,SGB_II-Bezieher,"(141, 142)",SGB_Ii-Bezieher
50,NOUN_PHRASE,62,P,ORG,3346603037676320710,2,222,46,Sparkasse_Köln,"(0, 1)",Sparkasse_Köln
63,NOUN_PHRASE,72,P,LOC,6030682432064371412,2,263,59,Stadt_Bonn,"(12, 13)",Stadt_Bonn
67,NOUN_PHRASE,74,P,PER,6030682432064371412,2,270,61,Martin_Nötzel,"(35, 36)",Martin_Nötzel
71,NOUN_PHRASE,76,P,LOC,6030682432064371412,2,277,63,Stadt_Bonn,"(85, 86)",Stadt_Bonn
78,NOUN_PHRASE,88,P,LOC,-4698369755082196440,2,327,72,Stadt_Bonn,"(10, 11)",Stadt_Bonn
90,NOUN_PHRASE,92,P,LOC,-805735878397083349,2,351,76,Stadt_Bonn,"(8, 9)",Stadt_Bonn


In [16]:
# based on Philipp Grawes approach on extracting and normalizing street names
STREET_NAME_LIST = [r'strasse$', r'straße$', r'str$', r'str.$', r'platz', r'gasse$', r'allee$', r'ufer$', r'weg$']
STREET_NAMES = re.compile(r'(' + '|'.join(STREET_NAME_LIST) + ')', re.IGNORECASE)
STREET_PATTERN = re.compile(r"str(\.|a(ss|ß)e)?\b", re.IGNORECASE)
SPECIAL_CHAR = re.compile(r'[^\w&\/]+')

def aggregate_streets(column):
    if column.name in {HASH, SENT_IDX, ENT_IDX}:
        return column.values[0]
    if column.name in {'tok_idx', NOUN_PHRASE, 'np'}:
        return tuple(column.values)
    if column.name == TEXT:
        return column.str.cat(sep='_')
    if column.name == TOKEN:
        street_candidate = False
        for k, token in column.iteritems():
            if re.search(STREET_NAMES, token):
                street_candidate = True
        if street_candidate:
            if len(column) == 1 and re.fullmatch(STREET_NAMES, column.values[0]):
                return False
            else:
                street_name = column.str.cat(sep=' ')
                street_name = STREET_PATTERN.sub('straße', street_name)
                street_name = SPECIAL_CHAR.sub('-', street_name)
                street_name = street_name.strip('-').title()
                return street_name
    return False

In [17]:
df_loc = df.loc[(df[ENT_IDX] > 0) & (df.POS != SPACE)]
df_loc = df_loc.groupby(ENT_IDX, as_index=False).agg(aggregate_streets)
df_loc[POS] = 'PROPN'
df_loc[ENT_IOB] = 'P'
df_loc[ENT_TYPE] = 'STREET'
df_loc[df_loc[TOKEN] != False]

,ent_idx,hash,tok_idx,sent_idx,text,token,POS,ent_iob,ent_type,noun_phrase
138,222,5102244154820621653,"(379, 380)",228,Öffentliche_Spielplätze,Öffentlich-Spielplatz,PROPN,P,STREET,"(709, 709)"
218,314,8887542040415661500,"(32,)",338,Vorrangstraßen,Vorrangstraße,PROPN,P,STREET,"(1154,)"
227,325,2090385250315026745,"(0,)",349,Arbeitsplatzproblem,Arbeitsplatzproblem,PROPN,P,STREET,"(1201,)"
235,335,-5949245709136083460,"(19, 20)",360,Auerberger_Allee,Auerberger-Allee,PROPN,P,STREET,"(1252, 1252)"
239,339,-5949245709136083460,"(111, 112)",369,Auerberger_Allee,Auerberger-Allee,PROPN,P,STREET,"(1277, 1277)"
242,342,-5949245709136083460,"(170, 171)",376,Berner_Straße,Berner-Straße,PROPN,P,STREET,"(1291, 1291)"
243,343,-5949245709136083460,"(217, 218)",378,Auerberger_Allee,Auerberger-Allee,PROPN,P,STREET,"(1299, 1299)"
244,344,-5949245709136083460,"(222, 223, 224)",378,Rundweg_Berner/Wiener_Straße,Rundweg-Berner/Wiener-Straße,PROPN,P,STREET,"(1300, 1301, 1301)"
447,592,4564080307884444329,"(199, 200, 201, 202)",622,P_&_R_Plätze,P-&-R-Platz,PROPN,P,STREET,"(2477, 0, 2478, 2478)"
485,632,4682684035684117078,"(198, 199)",646,"Museumsplatz),•_JazzTube",Museumsplatz-Jazztube,PROPN,P,STREET,"(2612, 2612)"
